[延伸閱讀 `__name__ == '__main__':`] from gemini  
Q : 請問執行Python module時、主程式中的`if __name__ == '__main__': main()`,其意義與用法為何?  

`if __name__ == '__main__'`: 是 Python 中一個極其重要且常見的結構，對於任何編寫模組（Module）的人來說都是必須掌握的。它主要用於區分一個 Python 檔案是在被導入（Imported）時執行，還是在直接執行時執行。

**<h3>`if __name__ == '__main__': main()` 的意義與用法</h3>**  
1. 概念解析：`__name__` 變數  
    Python 解釋器在執行任何程式碼檔案時，都會自動定義一些特殊的內建變數，其中一個就是 `__name__`。

    |執行方式	|`__name__ `的值|說明         |
    |:----------|:-------------|:------------|
    |直接執行	|`'__main__'`   |當你直接在命令列執行該檔案（例如：python my_script.py）時，Python 會將這個檔案視為主程式，所以賦予它特殊值 `'__main__'`。|
    |被導入	    |'my_script'	|當該檔案被其他檔案使用 `import my_script` 語句導入時，`__name__` 的值會是它的模組名稱（即檔案名稱，不含副檔名）。|  

2. **結構意義**
因此，`if __name__ == '__main__':` 這段程式碼的意義就是：
<h4 style='color:red'>「如果這個 Python 檔案是作為主程式直接啟動的，那麼才執行接下來的程式碼區塊。」</h4>  

3. **用法：隔離執行邏輯**  
這個結構最常見的用法是將檔案的主要執行邏輯放在一個名為 `main()` 的函式中，然後在條件判斷句內呼叫它：  
    ```
    # my_script.py

    def data_cleaning(df):
        """處理資料，這個函式會被其他模組重複使用"""
        # ... 清理邏輯 ...
        return cleaned_df

    def main():
        """主程式的執行邏輯：讀取、處理、輸出"""
        print("--- 正在作為主程式執行 ---")
        
        # 1. 讀取資料
        # df = pd.read_csv('data.csv') 
        
        # 2. 呼叫模組內定義的功能
        # cleaned_data = data_cleaning(df)
        
        # 3. 輸出結果
        print("主程序邏輯執行完畢。")

    # 核心判斷：
    if __name__ == '__main__':
        main()
    ```

**為什麼資料分析師需要這個結構？**  
這個結構對於模組化程式碼和重複使用性至關重要：

1. 防止模組被意外執行：

    * 假設你寫了一個名為 `analysis_tools.py` 的檔案，裡面定義了許多資料清理、計算等函式。

    * 如果沒有 `if __name__ == '__main__':`，當另一個腳本 `report_generator.py` 導入 `analysis_tools.py` 時，`analysis_tools.py` 裡所有的頂層程式碼（例如範例中的 `print("--- 正在作為主程式執行 ---")` ）都會被執行一次。

    * 有了這個結構，當 `report_generator.py` 導入它時，只有函式定義會被載入，`main()` 函式及其內容則不會執行。

2. 方便進行單元測試：

    * 你可以直接執行 `python my_script.py` 來測試該模組的全部功能。

    * 同時，其他模組可以導入 `my_script` 並只呼叫其中的特定函式（例如 `my_script.data_cleaning(df)`），而不會觸發整個 `main()` 流程。

簡而言之，這個結構提供了一個清晰的界線，讓你的 Python 檔案既可以作為可重複使用的工具箱（被導入），也可以作為一個獨立的應用程式（直接執行）。


[延伸閱讀 `__name__ == '__main__':`] from gemini - advance  
Q : 請問如何設計一個需要被其他腳本導入使用的 Python 模組嗎？  

設計一個 Python 模組，讓我們來看看一個標準的資料分析模組應該如何利用 `if __name__ == '__main__':` 結構來組織程式碼：

**標準資料分析模組的結構範例**
假設你的檔案名稱是 `data_prep.py`，專門用來執行資料的讀取、清理和初步轉換。

In [ ]:
# data_prep.py

import pandas as pd

# ----------------------------------------------------
# 1. 核心功能函式 (Functions - For Reusability)
#    - 這些函式是模組的「工具箱」，供其他腳本導入和使用
# ----------------------------------------------------

def load_data(file_path):
    """從 CSV 檔案載入資料，回傳 Pandas DataFrame。"""
    try:
        df = pd.read_csv(file_path)
        print(f"資料載入成功：{file_path}")
        return df
    except FileNotFoundError:
        print(f"錯誤：找不到檔案 {file_path}")
        return None

def clean_data(df):
    """執行資料清理：處理缺失值、移除重複項等。"""
    if df is None:
        return None
    
    # 示例清理邏輯
    df = df.dropna()
    print("資料清理完成：已移除所有缺失值。")
    return df

# ----------------------------------------------------
# 2. 主程式邏輯 (Main Execution Logic)
#    - 專門用於「直接執行」這個檔案時，進行端到端測試或獨立運行
# ----------------------------------------------------

def main():
    """定義當該檔案作為主程式執行時要執行的流程。"""
    
    # 設置檔案路徑
    data_file = 'sample_data.csv' # 假設這個檔案存在
    
    # 載入資料
    df = load_data(data_file)
    
    # 執行清理
    cleaned_df = clean_data(df)
    
    # 顯示初步結果或儲存
    if cleaned_df is not None:
        print("\n--- 清理後的資料概覽 ---")
        print(cleaned_df.head())
        # cleaned_df.to_csv('processed_data.csv', index=False)
        
# ----------------------------------------------------
# 3. 執行入口 (The Gatekeeper)
#    - 區分「被導入」還是「直接執行」
# ----------------------------------------------------

if __name__ == '__main__':
    # 只有當你在命令列執行 `python data_prep.py` 時，main() 才會被呼叫
    main()

# ----------------------------------------------------